In [76]:
import pandas as pd
import numpy as np
import datetime 
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [99]:
pd.set_option('precision', 2)

### Import summary tables 

In [100]:
empSt = pd.read_csv("../newDat/summary_stash/byState_employers.csv", dtype={'naics5': str})
emp = pd.read_csv("../newDat/summary_stash/byCounty_employers.csv", dtype={'fips':str, 'naics5': str})

work = pd.read_csv("../newDat/summary_stash/byCounty_worksite.csv", dtype={'fips':str, 'naics5': str})
workSt = pd.read_csv("../newDat/summary_stash/byState_worksite.csv", dtype={'naics5': str})

In [101]:
workSt.head()

,quarter,result,state,app_count,worker_count,min_yr_sal,avg_yr_sal,max_yr_sal,min_pw,avg_pw,max_pw,avg_decdur,naics5,job,workcity,samest,ssst
0,2015Q1,CERTIFIED,AK,25,27,31637.000,113003.000,580653.000,31637.000,81200.000,193960.000,6,61131,PROGRAMMER ANALYST,ANCHORAGE-AK,16,64.000
1,2015Q1,CERTIFIED,AL,165,302,20800.000,76074.000,237474.000,17618.000,63484.000,187199.000,6,54151,PROGRAMMER ANALYST,MONTGOMERY-AL,68,41.210
2,2015Q1,CERTIFIED,AR,374,659,29059.000,74284.000,200000.000,22277.000,61187.000,187200.000,5,54151,TECHNOLOGY LEAD - US,BENTONVILLE-AR,89,23.800
3,2015Q1,CERTIFIED,AZ,1118,3319,31125.000,80537.000,420000.000,24482.000,68552.000,210995.000,5,54151,PROGRAMMER ANALYST,PHOENIX-AZ,239,21.380
4,2015Q1,CERTIFIED,CA,13902,29419,18720.000,101535.000,580000.000,18720.000,87853.000,249995.000,5,54151,SOFTWARE ENGINEER,SAN FRANCISCO-CA,7949,57.180


In [102]:
empSt.columns = ['QRT', 'RST', 'STATE', 'APP', 'WORKER', 
                 'MINSAL', 'AVGSAL', 'MAXSAL', 'MINPW', 'AVGPW', 'MAXPW', 
                 'AVGDUR', 'TOPNAIC5', 'TOPJOB', 'TOPCITY', 'INSTATE', 'PCTINSTATE']

In [103]:
emp.columns = ['QRT', 'RST', 'CNTY', 'APP', 'WORKER', 
                 'MINSAL', 'AVGSAL', 'MAXSAL', 'MINPW', 'AVGPW', 'MAXPW', 
                 'AVGDUR', 'TOPNAIC5', 'TOPJOB', 'TOPCITY', 'INSTATE', 'PCTINSTATE']

In [104]:
workSt.columns = ['QRT', 'RST', 'STATE', 'APP', 'WORKER', 
                 'MINSAL', 'AVGSAL', 'MAXSAL', 'MINPW', 'AVGPW', 'MAXPW', 
                 'AVGDUR', 'TOPNAIC5', 'TOPJOB', 'TOPCITY', 'INSTATE', 'PCTINSTATE']

In [105]:
work.columns = ['QRT', 'RST', 'CNTY', 'APP', 'WORKER', 
                 'MINSAL', 'AVGSAL', 'MAXSAL', 'MINPW', 'AVGPW', 'MAXPW', 
                 'AVGDUR', 'TOPNAIC5', 'TOPJOB', 'TOPCITY', 'INSTATE', 'PCTINSTATE']


#shedding cols

In [106]:
empSt = empSt[['QRT', 'RST', 'STATE', 'APP', 'WORKER', 
                'AVGSAL', 'AVGPW', 'TOPNAIC5', 'TOPJOB', 'TOPCITY', 'PCTINSTATE']]

In [107]:
emp = emp[['QRT', 'RST', 'CNTY', 'APP', 'WORKER', 
                'AVGSAL', 'AVGPW', 'TOPNAIC5', 'TOPJOB', 'TOPCITY', 'PCTINSTATE']]

In [108]:
workSt = workSt[['QRT', 'RST', 'STATE', 'APP', 'WORKER', 
                'AVGSAL', 'AVGPW', 'TOPNAIC5', 'TOPJOB', 'TOPCITY', 'PCTINSTATE']]

In [109]:
work = work[['QRT', 'RST', 'CNTY', 'APP', 'WORKER', 
                'AVGSAL', 'AVGPW', 'TOPNAIC5', 'TOPJOB', 'TOPCITY', 'PCTINSTATE']]


## Process: 

#### Wanted Outputs: 1 state geojson, 1 county geojson, 1 cities geojson

Stretch goal: also include 'all' results category. But this will involve going back to the vvvdata.csv file. 

The goal is to have index of geographies that will include prefixed columns as these: 

    [15Q1_DENIED_appcount, 15Q1_DENIED_workercount ... etc]
    
    Take this time to clean up column names, get rid of unnecessary columns
    
    
1. Produce separate dataframes for each Quarter_Status (should be 13*5) for counties and for states. 

In [110]:
quarter = {'2015Q1' : '15Q1', '2015Q2' : '15Q2', '2015Q3' : '15Q3', '2015Q4' : '15Q4', 
           '2016Q1' : '16Q1', '2016Q2' : '16Q2', '2016Q3' : '16Q3', '2016Q4' : '16Q4',
           '2017Q1' : '17Q1', '2017Q2' : '17Q2', '2017Q3' : '17Q3', '2017Q4' : '17Q4',
           '2018Q1' : '18Q1'}
result = {'CERTIFIED' : 'CERT', 'CERTIFIED-WITHDRAWN' : 'CERTWD', 'DENIED': 'DEN', 'WITHDRAWN' : 'WD'}

In [111]:
def remap_categories(table, colList, namedictList): 
    for col in colList: 
        for ndict in namedictList: 
            for cat in ndict.keys(): 
                table.loc[table[col] == cat, col] = ndict[cat]
                print (ndict[cat])
        table[col] = table[col].astype('category')
    print (" > success")

In [112]:
remap_categories(emp, ['QRT', 'RST'], [quarter, result])
remap_categories(empSt, ['QRT', 'RST'], [quarter, result])
remap_categories(work, ['QRT', 'RST'], [quarter, result])
remap_categories(workSt, ['QRT', 'RST'], [quarter, result])

15Q1
15Q2
15Q3
15Q4
16Q1
16Q2
16Q3
16Q4
17Q1
17Q2
17Q3
17Q4
18Q1
CERT
CERTWD
DEN
WD
15Q1
15Q2
15Q3
15Q4
16Q1
16Q2
16Q3
16Q4
17Q1
17Q2
17Q3
17Q4
18Q1
CERT
CERTWD
DEN
WD
 > success
15Q1
15Q2
15Q3
15Q4
16Q1
16Q2
16Q3
16Q4
17Q1
17Q2
17Q3
17Q4
18Q1
CERT
CERTWD
DEN
WD
15Q1
15Q2
15Q3
15Q4
16Q1
16Q2
16Q3
16Q4
17Q1
17Q2
17Q3
17Q4
18Q1
CERT
CERTWD
DEN
WD
 > success
15Q1
15Q2
15Q3
15Q4
16Q1
16Q2
16Q3
16Q4
17Q1
17Q2
17Q3
17Q4
18Q1
CERT
CERTWD
DEN
WD
15Q1
15Q2
15Q3
15Q4
16Q1
16Q2
16Q3
16Q4
17Q1
17Q2
17Q3
17Q4
18Q1
CERT
CERTWD
DEN
WD
 > success
15Q1
15Q2
15Q3
15Q4
16Q1
16Q2
16Q3
16Q4
17Q1
17Q2
17Q3
17Q4
18Q1
CERT
CERTWD
DEN
WD
15Q1
15Q2
15Q3
15Q4
16Q1
16Q2
16Q3
16Q4
17Q1
17Q2
17Q3
17Q4
18Q1
CERT
CERTWD
DEN
WD
 > success


In [117]:
emp.loc[emp['CNTY'] == '12025', 'CNTY'] = '12086'
emp[emp['CNTY'] == '12086'].head()

,QRT,RST,CNTY,APP,WORKER,AVGSAL,AVGPW,TOPNAIC5,TOPJOB,TOPCITY,PCTINSTATE
130,15Q1,CERT,12086,250,251,88813.000,68932.000,54151,PROGRAMMER ANALYST,MIAMI-FL,76.000
1020,15Q1,CERTWD,12086,16,18,87773.000,57890.000,61131,INTERNAL MEDICINE RESIDENT,MIAMI-FL,87.500
1376,15Q1,DEN,12086,13,14,85809.000,286807.000,61169,TEACHER ASSISTANT,MIAMI-FL,92.310
1648,15Q1,WD,12086,10,10,69168.000,59046.000,32799,TRADER,MIAMI-FL,90.000
2054,15Q2,CERT,12086,3,3,47692.000,47486.000,52311,MARKETING & RETAIL SPECIALIST,MIAMI-FL,100.000


In [119]:
work[work['CNTY'] == '12086'].head()

,QRT,RST,CNTY,APP,WORKER,AVGSAL,AVGPW,TOPNAIC5,TOPJOB,TOPCITY,PCTINSTATE
159,15Q1,CERT,12086,342,542,88131.000,69504.000,54151,PROGRAMMER ANALYST,MIAMI-FL,57.890
1375,15Q1,CERTWD,12086,41,42,88987.000,71218.000,54151,PROGRAMMER/ANALYST,MIAMI-FL,29.270
1854,15Q1,DEN,12086,13,14,93211.000,292113.000,51512,TEACHER ASSISTANT,MIAMI-FL,84.620
2163,15Q1,WD,12086,19,23,114237.000,62485.000,54191,"SOFTWARE DEVELOPERS, APPLCIATIONS",MIAMI-FL,47.370
2678,15Q2,CERT,12086,1847,2940,71381.000,60661.000,54151,PROGRAMMER ANALYST,MIAMI-FL,70.760


In [ ]:
work.loc[work['CNTY'] == '12025', 'CNTY'] = '12086'
work[work['CNTY'] == '12086'].head()

In [120]:
## exporting for carto 
## will be joined on qgis first
# no prefix - will be filtered by SQL
empSt.to_csv("../newDat/carto/byState_employer.csv", index = False)
emp.to_csv("../newDat/carto/byCounty_employer.csv", index = False)
workSt.to_csv("../newDat/carto/byState_worksite.csv", index = False)
work.to_csv("../newDat/carto/byCounty_worksite.csv", index = False)

def categorize(table, colList):
    """
    Take the input table and categorize the columns that are included in the colList array
    """
    
    for col in colList: 
        table[col] = table[col].astype('category')
        print ("colums cateogrized: ", col)

categorize(emp, ['quarter', 'result'])
categorize(empSt, ['quarter', 'result'])
categorize(work, ['quarter', 'result'])
categorize(workSt, ['quarter', 'result'])

In [16]:
emp.dtypes

QRT           category
RST           category
CNTY            object
APP              int64
WORKER           int64
AVGSAL         float64
AVGPW          float64
TOPNAIC5        object
TOPJOB          object
TOPCITY         object
PCTINSTATE     float64
dtype: object

In [17]:
def list_dfs (inputDat, dat_prefiex): 
    dataframes = []
    results = inputDat['RST'].unique()
    periods = inputDat['QRT'].unique()
    
    for p in periods:
        for r in results: 
            output_name = dat_prefix + "_" + p + "_" + r
            loc = len(dataframes)
            dataframes.append(output_name)
            #globals()['output_name'] = inputDat[(inputDat['result'] == r) & (inputDat['quarter'] == p)].copy()
    return dataframes

In [18]:
def list_dfs2 (inputDat): 
    dataframes = []
    results = inputDat['RST'].unique()
    periods = inputDat['QRT'].unique()
    
    for p in periods:
        for r in results: 
            output_name = p + "_" + r
            loc = len(dataframes)
            dataframes.append(output_name)
            #globals()['output_name'] = inputDat[(inputDat['result'] == r) & (inputDat['quarter'] == p)].copy()
    return dataframes

In [19]:
#emplist = list_dfs(emp, "emp")
#empstlist = list_dfs(empSt, "empst")
#worklist = list_dfs(work, "work")
#workstlist = list_dfs(workSt, "workst")
dflist = list_dfs2(emp)

In [20]:
dflist

['15Q1_CERT',
 '15Q1_CERTWD',
 '15Q1_DEN',
 '15Q1_WD',
 '15Q2_CERT',
 '15Q2_CERTWD',
 '15Q2_DEN',
 '15Q2_WD',
 '15Q3_CERT',
 '15Q3_CERTWD',
 '15Q3_DEN',
 '15Q3_WD',
 '15Q4_CERT',
 '15Q4_CERTWD',
 '15Q4_DEN',
 '15Q4_WD',
 '16Q1_CERT',
 '16Q1_CERTWD',
 '16Q1_DEN',
 '16Q1_WD',
 '16Q2_CERT',
 '16Q2_CERTWD',
 '16Q2_DEN',
 '16Q2_WD',
 '16Q3_CERT',
 '16Q3_CERTWD',
 '16Q3_DEN',
 '16Q3_WD',
 '16Q4_CERT',
 '16Q4_CERTWD',
 '16Q4_DEN',
 '16Q4_WD',
 '17Q1_CERT',
 '17Q1_CERTWD',
 '17Q1_DEN',
 '17Q1_WD',
 '17Q2_CERT',
 '17Q2_CERTWD',
 '17Q2_DEN',
 '17Q2_WD',
 '17Q3_CERT',
 '17Q3_CERTWD',
 '17Q3_DEN',
 '17Q3_WD',
 '17Q4_CERT',
 '17Q4_CERTWD',
 '17Q4_DEN',
 '17Q4_WD',
 '18Q1_CERT',
 '18Q1_CERTWD',
 '18Q1_DEN',
 '18Q1_WD']

In [21]:
datStr = ["emp", "empSt", "work", "workSt"] 
dats = [emp, empSt, work, workSt]


In [22]:
datStr[0]

'emp'

In [23]:
def tinitize(inputDat, dfname):
    """
    Prepares datasets into smaller dataframes that uses geography as the single index
    The smaller dataframes will be specific to each quarter-result combo.
    """
    
    dfname_parsed = dfname.split("_")
    qrt = dfname_parsed[0]
    rst = dfname_parsed[1]
    #print (dfname_parsed, qrt)
    
    outputdf = inputDat[(inputDat['QRT'] == qrt) & (inputDat['RST'] == rst)]
    return outputdf
    
    

In [24]:
for bi, bval in enumerate(datStr):
    for i, val in enumerate(dflist):
        #print (datStr[bi], str(val), dats[bi], str(val))
        globals()[datStr[bi] + "_" + str(val)] = tinitize(dats[bi], str(val))


In [25]:
work_17Q3_CERT.head()

,QRT,RST,CNTY,APP,WORKER,AVGSAL,AVGPW,TOPNAIC5,TOPJOB,TOPCITY,PCTINSTATE
29258,17Q3,CERT,01001,2,2,95000.000,187200.000,62111,NEPHROLOGIST,PRATTVILLE-AL,100.000
29259,17Q3,CERT,01003,1,1,42450.000,51210.000,54151,HYPERION SUPPORT ANALYST,SPANISH FORT-AL,100.000
29260,17Q3,CERT,01011,3,21,89293.000,96893.000,54151,IT SUPPORT SPECIALIST,TROY-AL,33.330
29261,17Q3,CERT,01013,2,2,95000.000,163322.000,62111,NEPHROLOGIST,GREENVILLE-AL,100.000
29262,17Q3,CERT,01015,8,8,71856.000,79484.000,62111,HOSPITALIST PHYSICIAN,ANNISTON-AL,62.500


In [26]:
tablels = pd.DataFrame({'format': dflist})
tablels.head()

,format
0,15Q1_CERT
1,15Q1_CERTWD
2,15Q1_DEN
3,15Q1_WD
4,15Q2_CERT


In [27]:
tablels['emp'] = tablels['format'].apply(lambda x: 'emp_' + x)
tablels['empSt'] = tablels['format'].apply(lambda x: 'empSt_' + x)
tablels['work'] = tablels['format'].apply(lambda x: 'work_' + x)
tablels['workSt'] = tablels['format'].apply(lambda x: 'workSt_' + x)

In [28]:
empls = ", ".join(tablels['emp'])
empStls = ", ".join(tablels['empSt'])
workls = ", ".join(tablels['work'])
workStls = ", ".join(tablels['workSt'])

In [29]:
empls

'emp_15Q1_CERT, emp_15Q1_CERTWD, emp_15Q1_DEN, emp_15Q1_WD, emp_15Q2_CERT, emp_15Q2_CERTWD, emp_15Q2_DEN, emp_15Q2_WD, emp_15Q3_CERT, emp_15Q3_CERTWD, emp_15Q3_DEN, emp_15Q3_WD, emp_15Q4_CERT, emp_15Q4_CERTWD, emp_15Q4_DEN, emp_15Q4_WD, emp_16Q1_CERT, emp_16Q1_CERTWD, emp_16Q1_DEN, emp_16Q1_WD, emp_16Q2_CERT, emp_16Q2_CERTWD, emp_16Q2_DEN, emp_16Q2_WD, emp_16Q3_CERT, emp_16Q3_CERTWD, emp_16Q3_DEN, emp_16Q3_WD, emp_16Q4_CERT, emp_16Q4_CERTWD, emp_16Q4_DEN, emp_16Q4_WD, emp_17Q1_CERT, emp_17Q1_CERTWD, emp_17Q1_DEN, emp_17Q1_WD, emp_17Q2_CERT, emp_17Q2_CERTWD, emp_17Q2_DEN, emp_17Q2_WD, emp_17Q3_CERT, emp_17Q3_CERTWD, emp_17Q3_DEN, emp_17Q3_WD, emp_17Q4_CERT, emp_17Q4_CERTWD, emp_17Q4_DEN, emp_17Q4_WD, emp_18Q1_CERT, emp_18Q1_CERTWD, emp_18Q1_DEN, emp_18Q1_WD'

In [30]:
emp_dflist = [emp_15Q1_CERT, emp_15Q1_CERTWD, emp_15Q1_DEN, emp_15Q1_WD, emp_15Q2_CERT, emp_15Q2_CERTWD, emp_15Q2_DEN, emp_15Q2_WD, emp_15Q3_CERT, emp_15Q3_CERTWD, emp_15Q3_DEN, emp_15Q3_WD, emp_15Q4_CERT, emp_15Q4_CERTWD, emp_15Q4_DEN, emp_15Q4_WD, emp_16Q1_CERT, emp_16Q1_CERTWD, emp_16Q1_DEN, emp_16Q1_WD, emp_16Q2_CERT, emp_16Q2_CERTWD, emp_16Q2_DEN, emp_16Q2_WD, emp_16Q3_CERT, emp_16Q3_CERTWD, emp_16Q3_DEN, emp_16Q3_WD, emp_16Q4_CERT, emp_16Q4_CERTWD, emp_16Q4_DEN, emp_16Q4_WD, emp_17Q1_CERT, emp_17Q1_CERTWD, emp_17Q1_DEN, emp_17Q1_WD, emp_17Q2_CERT, emp_17Q2_CERTWD, emp_17Q2_DEN, emp_17Q2_WD, emp_17Q3_CERT, emp_17Q3_CERTWD, emp_17Q3_DEN, emp_17Q3_WD, emp_17Q4_CERT, emp_17Q4_CERTWD, emp_17Q4_DEN, emp_17Q4_WD, emp_18Q1_CERT, emp_18Q1_CERTWD, emp_18Q1_DEN, emp_18Q1_WD]

In [31]:
empStls

'empSt_15Q1_CERT, empSt_15Q1_CERTWD, empSt_15Q1_DEN, empSt_15Q1_WD, empSt_15Q2_CERT, empSt_15Q2_CERTWD, empSt_15Q2_DEN, empSt_15Q2_WD, empSt_15Q3_CERT, empSt_15Q3_CERTWD, empSt_15Q3_DEN, empSt_15Q3_WD, empSt_15Q4_CERT, empSt_15Q4_CERTWD, empSt_15Q4_DEN, empSt_15Q4_WD, empSt_16Q1_CERT, empSt_16Q1_CERTWD, empSt_16Q1_DEN, empSt_16Q1_WD, empSt_16Q2_CERT, empSt_16Q2_CERTWD, empSt_16Q2_DEN, empSt_16Q2_WD, empSt_16Q3_CERT, empSt_16Q3_CERTWD, empSt_16Q3_DEN, empSt_16Q3_WD, empSt_16Q4_CERT, empSt_16Q4_CERTWD, empSt_16Q4_DEN, empSt_16Q4_WD, empSt_17Q1_CERT, empSt_17Q1_CERTWD, empSt_17Q1_DEN, empSt_17Q1_WD, empSt_17Q2_CERT, empSt_17Q2_CERTWD, empSt_17Q2_DEN, empSt_17Q2_WD, empSt_17Q3_CERT, empSt_17Q3_CERTWD, empSt_17Q3_DEN, empSt_17Q3_WD, empSt_17Q4_CERT, empSt_17Q4_CERTWD, empSt_17Q4_DEN, empSt_17Q4_WD, empSt_18Q1_CERT, empSt_18Q1_CERTWD, empSt_18Q1_DEN, empSt_18Q1_WD'

In [32]:
empSt_dflist = [empSt_15Q1_CERT, empSt_15Q1_CERTWD, empSt_15Q1_DEN, empSt_15Q1_WD, empSt_15Q2_CERT, empSt_15Q2_CERTWD, empSt_15Q2_DEN, empSt_15Q2_WD, empSt_15Q3_CERT, empSt_15Q3_CERTWD, empSt_15Q3_DEN, empSt_15Q3_WD, empSt_15Q4_CERT, empSt_15Q4_CERTWD, empSt_15Q4_DEN, empSt_15Q4_WD, empSt_16Q1_CERT, empSt_16Q1_CERTWD, empSt_16Q1_DEN, empSt_16Q1_WD, empSt_16Q2_CERT, empSt_16Q2_CERTWD, empSt_16Q2_DEN, empSt_16Q2_WD, empSt_16Q3_CERT, empSt_16Q3_CERTWD, empSt_16Q3_DEN, empSt_16Q3_WD, empSt_16Q4_CERT, empSt_16Q4_CERTWD, empSt_16Q4_DEN, empSt_16Q4_WD, empSt_17Q1_CERT, empSt_17Q1_CERTWD, empSt_17Q1_DEN, empSt_17Q1_WD, empSt_17Q2_CERT, empSt_17Q2_CERTWD, empSt_17Q2_DEN, empSt_17Q2_WD, empSt_17Q3_CERT, empSt_17Q3_CERTWD, empSt_17Q3_DEN, empSt_17Q3_WD, empSt_17Q4_CERT, empSt_17Q4_CERTWD, empSt_17Q4_DEN, empSt_17Q4_WD, empSt_18Q1_CERT, empSt_18Q1_CERTWD, empSt_18Q1_DEN, empSt_18Q1_WD]


In [33]:
workls

'work_15Q1_CERT, work_15Q1_CERTWD, work_15Q1_DEN, work_15Q1_WD, work_15Q2_CERT, work_15Q2_CERTWD, work_15Q2_DEN, work_15Q2_WD, work_15Q3_CERT, work_15Q3_CERTWD, work_15Q3_DEN, work_15Q3_WD, work_15Q4_CERT, work_15Q4_CERTWD, work_15Q4_DEN, work_15Q4_WD, work_16Q1_CERT, work_16Q1_CERTWD, work_16Q1_DEN, work_16Q1_WD, work_16Q2_CERT, work_16Q2_CERTWD, work_16Q2_DEN, work_16Q2_WD, work_16Q3_CERT, work_16Q3_CERTWD, work_16Q3_DEN, work_16Q3_WD, work_16Q4_CERT, work_16Q4_CERTWD, work_16Q4_DEN, work_16Q4_WD, work_17Q1_CERT, work_17Q1_CERTWD, work_17Q1_DEN, work_17Q1_WD, work_17Q2_CERT, work_17Q2_CERTWD, work_17Q2_DEN, work_17Q2_WD, work_17Q3_CERT, work_17Q3_CERTWD, work_17Q3_DEN, work_17Q3_WD, work_17Q4_CERT, work_17Q4_CERTWD, work_17Q4_DEN, work_17Q4_WD, work_18Q1_CERT, work_18Q1_CERTWD, work_18Q1_DEN, work_18Q1_WD'

In [34]:
work_dflist = [work_15Q1_CERT, work_15Q1_CERTWD, work_15Q1_DEN, work_15Q1_WD, work_15Q2_CERT, work_15Q2_CERTWD, work_15Q2_DEN, work_15Q2_WD, work_15Q3_CERT, work_15Q3_CERTWD, work_15Q3_DEN, work_15Q3_WD, work_15Q4_CERT, work_15Q4_CERTWD, work_15Q4_DEN, work_15Q4_WD, work_16Q1_CERT, work_16Q1_CERTWD, work_16Q1_DEN, work_16Q1_WD, work_16Q2_CERT, work_16Q2_CERTWD, work_16Q2_DEN, work_16Q2_WD, work_16Q3_CERT, work_16Q3_CERTWD, work_16Q3_DEN, work_16Q3_WD, work_16Q4_CERT, work_16Q4_CERTWD, work_16Q4_DEN, work_16Q4_WD, work_17Q1_CERT, work_17Q1_CERTWD, work_17Q1_DEN, work_17Q1_WD, work_17Q2_CERT, work_17Q2_CERTWD, work_17Q2_DEN, work_17Q2_WD, work_17Q3_CERT, work_17Q3_CERTWD, work_17Q3_DEN, work_17Q3_WD, work_17Q4_CERT, work_17Q4_CERTWD, work_17Q4_DEN, work_17Q4_WD, work_18Q1_CERT, work_18Q1_CERTWD, work_18Q1_DEN, work_18Q1_WD]


In [35]:
workStls

'workSt_15Q1_CERT, workSt_15Q1_CERTWD, workSt_15Q1_DEN, workSt_15Q1_WD, workSt_15Q2_CERT, workSt_15Q2_CERTWD, workSt_15Q2_DEN, workSt_15Q2_WD, workSt_15Q3_CERT, workSt_15Q3_CERTWD, workSt_15Q3_DEN, workSt_15Q3_WD, workSt_15Q4_CERT, workSt_15Q4_CERTWD, workSt_15Q4_DEN, workSt_15Q4_WD, workSt_16Q1_CERT, workSt_16Q1_CERTWD, workSt_16Q1_DEN, workSt_16Q1_WD, workSt_16Q2_CERT, workSt_16Q2_CERTWD, workSt_16Q2_DEN, workSt_16Q2_WD, workSt_16Q3_CERT, workSt_16Q3_CERTWD, workSt_16Q3_DEN, workSt_16Q3_WD, workSt_16Q4_CERT, workSt_16Q4_CERTWD, workSt_16Q4_DEN, workSt_16Q4_WD, workSt_17Q1_CERT, workSt_17Q1_CERTWD, workSt_17Q1_DEN, workSt_17Q1_WD, workSt_17Q2_CERT, workSt_17Q2_CERTWD, workSt_17Q2_DEN, workSt_17Q2_WD, workSt_17Q3_CERT, workSt_17Q3_CERTWD, workSt_17Q3_DEN, workSt_17Q3_WD, workSt_17Q4_CERT, workSt_17Q4_CERTWD, workSt_17Q4_DEN, workSt_17Q4_WD, workSt_18Q1_CERT, workSt_18Q1_CERTWD, workSt_18Q1_DEN, workSt_18Q1_WD'

In [36]:
workSt_dflist = [workSt_15Q1_CERT, workSt_15Q1_CERTWD, workSt_15Q1_DEN, workSt_15Q1_WD, workSt_15Q2_CERT, workSt_15Q2_CERTWD, workSt_15Q2_DEN, workSt_15Q2_WD, workSt_15Q3_CERT, workSt_15Q3_CERTWD, workSt_15Q3_DEN, workSt_15Q3_WD, workSt_15Q4_CERT, workSt_15Q4_CERTWD, workSt_15Q4_DEN, workSt_15Q4_WD, workSt_16Q1_CERT, workSt_16Q1_CERTWD, workSt_16Q1_DEN, workSt_16Q1_WD, workSt_16Q2_CERT, workSt_16Q2_CERTWD, workSt_16Q2_DEN, workSt_16Q2_WD, workSt_16Q3_CERT, workSt_16Q3_CERTWD, workSt_16Q3_DEN, workSt_16Q3_WD, workSt_16Q4_CERT, workSt_16Q4_CERTWD, workSt_16Q4_DEN, workSt_16Q4_WD, workSt_17Q1_CERT, workSt_17Q1_CERTWD, workSt_17Q1_DEN, workSt_17Q1_WD, workSt_17Q2_CERT, workSt_17Q2_CERTWD, workSt_17Q2_DEN, workSt_17Q2_WD, workSt_17Q3_CERT, workSt_17Q3_CERTWD, workSt_17Q3_DEN, workSt_17Q3_WD, workSt_17Q4_CERT, workSt_17Q4_CERTWD, workSt_17Q4_DEN, workSt_17Q4_WD, workSt_18Q1_CERT, workSt_18Q1_CERTWD, workSt_18Q1_DEN, workSt_18Q1_WD]


In [37]:
tablels.head()

,format,emp,empSt,work,workSt
0,15Q1_CERT,emp_15Q1_CERT,empSt_15Q1_CERT,work_15Q1_CERT,workSt_15Q1_CERT
1,15Q1_CERTWD,emp_15Q1_CERTWD,empSt_15Q1_CERTWD,work_15Q1_CERTWD,workSt_15Q1_CERTWD
2,15Q1_DEN,emp_15Q1_DEN,empSt_15Q1_DEN,work_15Q1_DEN,workSt_15Q1_DEN
3,15Q1_WD,emp_15Q1_WD,empSt_15Q1_WD,work_15Q1_WD,workSt_15Q1_WD
4,15Q2_CERT,emp_15Q2_CERT,empSt_15Q2_CERT,work_15Q2_CERT,workSt_15Q2_CERT


In [38]:
## adding prefixes to col names
#df.columns = [str(col) + '_x' for col in df.columns]

for i, df in enumerate(emp_dflist):
    
    df.drop(columns = ['QRT', 'RST'], inplace = True)
    df.columns = [tablels.loc[i, 'emp'] + '_' + str(col) for col in df.columns]
    geog = tablels.loc[i, 'emp'] + '_CNTY'
    df.rename(columns = {geog : 'CNTY'}, inplace = True)

for i, df in enumerate(empSt_dflist):
    
    df.drop(columns = ['QRT', 'RST'], inplace = True)
    df.columns = [tablels.loc[i, 'empSt'] + '_' + str(col) for col in df.columns] 
    geog = tablels.loc[i, 'empSt'] + '_STATE'
    df.rename(columns = {geog : 'STATE'}, inplace = True)

for i, df in enumerate(work_dflist):
    
    df.drop(columns = ['QRT', 'RST'], inplace = True)
    df.columns = [tablels.loc[i, 'work'] + '_' + str(col) for col in df.columns]
    geog = tablels.loc[i, 'work'] + '_CNTY'
    df.rename(columns = {geog : 'CNTY'}, inplace = True)

for i, df in enumerate(workSt_dflist):
    
    df.drop(columns = ['QRT', 'RST'], inplace = True)
    df.columns = [tablels.loc[i, 'workSt'] + '_' + str(col) for col in df.columns]
    geog = tablels.loc[i, 'workSt'] + '_STATE'
    df.rename(columns = {geog : 'STATE'}, inplace = True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A v

In [39]:
emp_16Q1_CERT.head()

,CNTY,emp_16Q1_CERT_APP,emp_16Q1_CERT_WORKER,emp_16Q1_CERT_AVGSAL,emp_16Q1_CERT_AVGPW,emp_16Q1_CERT_TOPNAIC5,emp_16Q1_CERT_TOPJOB,emp_16Q1_CERT_TOPCITY,emp_16Q1_CERT_PCTINSTATE
9420,01013,1,1,46500.000,49837.000,56299,ACCOUNTANT,GREENVILLE-AL,100.000
9421,01017,3,3,24117.000,44706.000,3363,PRODUCTION MANAGEMENT ANALYST,AUBURN-AL,100.000
9422,01033,1,1,167864.000,130728.000,62111,FAMILY MEDICINE PHYSICIAN,RUSSELLVILLE-AL,100.000
9423,01055,1,1,182500.000,275227.000,62111,PULMONARY PHYSICIAN,GADSDEN-AL,100.000
9424,01073,23,33,52574.000,72761.000,54171.0,ASSOCIATE RESEARCH CHEMIST,BIRMINGHAM-AL,78.260


In [40]:
empSt_15Q4_DEN.head()

,STATE,empSt_15Q4_DEN_APP,empSt_15Q4_DEN_WORKER,empSt_15Q4_DEN_AVGSAL,empSt_15Q4_DEN_AVGPW,empSt_15Q4_DEN_TOPNAIC5,empSt_15Q4_DEN_TOPJOB,empSt_15Q4_DEN_TOPCITY,empSt_15Q4_DEN_PCTINSTATE
734,AK,4,4,72142.000,68973.000,62134,SPEECH LANGUAGE PATHOLOGIST,SOLDOTNA-AK,75.000
735,AL,13,13,189718.000,159151.000,62211,RADIOLOGIST,BIRMINGHAM-AL,76.920
736,AR,5,5,58926.000,56408.000,54151,PROGRAMMER ANALYST,LITTLE ROCK-AR,40.000
737,AZ,16,16,81093.000,68359.000,54151,INTERNET SALES MANAGER,PHOENIX-AZ,75.000
738,CA,290,483,98473.000,1425825.000,54151.0,SOFTWARE ENGINEER,LOS ANGELES-CA,79.660


In [41]:
work_18Q1_WD.head()

,CNTY,work_18Q1_WD_APP,work_18Q1_WD_WORKER,work_18Q1_WD_AVGSAL,work_18Q1_WD_AVGPW,work_18Q1_WD_TOPNAIC5,work_18Q1_WD_TOPJOB,work_18Q1_WD_TOPCITY,work_18Q1_WD_PCTINSTATE
36709,01073,4,4,69666.000,100630.000,62149,NEPHROLOGIST,BIRMINGHAM-AL,25.000
36710,01101,1,1,29193.000,58386.000,5311,FINANCIAL ANALYST,MONTGOMERY-AL,100.000
36711,04003,1,1,30000.000,58635.000,11531,AGRONOMIST,PEARCE-AZ,100.000
36712,04013,45,48,72179.000,79346.000,54151,TECHNICAL LEAD,PHOENIX-AZ,13.330
36713,04019,9,18,46583.000,59762.000,61131,ASSISTANT PROFESSOR,TUCSON-AZ,77.780


In [42]:
len(workSt_17Q2_CERT.columns)

9

In [43]:
len(tablels)

52

In [44]:
from functools import reduce

In [45]:
emp_final = reduce(lambda left,right: pd.merge(left,right,how = 'outer', on='CNTY'), emp_dflist)
empSt_final = reduce(lambda left,right: pd.merge(left,right, how = 'outer', on='STATE'), empSt_dflist)
work_final = reduce(lambda left,right: pd.merge(left,right, how = 'outer', on='CNTY'), work_dflist)
workSt_final = reduce(lambda left,right: pd.merge(left,right, how = 'outer', on='STATE'), workSt_dflist)

In [52]:
empSt_final.to_csv("../newDat/geog/empSt_final.csv")
emp_final.to_csv("../newDat/geog/emp_final.csv")
work_final.to_csv("../newDat/geog/work_final.csv")
workSt_final.to_csv("../newDat/geog/workSt_final.csv")

In [53]:
import geopandas as gpd

In [54]:
counties = "../newDat/geog/us_counties.shp"
counties = gpd.read_file(counties)

In [55]:
states = "../newDat/geog/us_states.shp"
states = gpd.read_file(states)

In [56]:
statesj = states[['STUSPS', 'NAME', 'geometry']].copy()
statesj.columns = ['STATE', 'NAME', 'geometry']
statesj.head()

,STATE,NAME,geometry
0,AL,Alabama,(POLYGON ((-9800984.638293844 3548540.26655303...
1,AK,Alaska,(POLYGON ((-16724873.9563926 8620082.956959438...
2,AZ,Arizona,POLYGON ((-12779442.25478922 3818472.178664201...
3,CO,Colorado,POLYGON ((-12140531.82974559 4637779.187709421...
4,FL,Florida,(POLYGON ((-8989231.779480213 2840266.22056617...


In [57]:
byState_employer = pd.merge(statesj, empSt_final, how = 'left', on = 'STATE')
byState_employer = byState_employer.to_crs({'init': 'epsg:4326'})
byState_employer.head()

,STATE,NAME,geometry,empSt_15Q1_CERT_APP,empSt_15Q1_CERT_WORKER,empSt_15Q1_CERT_AVGSAL,empSt_15Q1_CERT_AVGPW,empSt_15Q1_CERT_TOPNAIC5,empSt_15Q1_CERT_TOPJOB,empSt_15Q1_CERT_TOPCITY,empSt_15Q1_CERT_PCTINSTATE,empSt_15Q1_CERTWD_APP,empSt_15Q1_CERTWD_WORKER,empSt_15Q1_CERTWD_AVGSAL,empSt_15Q1_CERTWD_AVGPW,empSt_15Q1_CERTWD_TOPNAIC5,empSt_15Q1_CERTWD_TOPJOB,empSt_15Q1_CERTWD_TOPCITY,empSt_15Q1_CERTWD_PCTINSTATE,empSt_15Q1_DEN_APP,empSt_15Q1_DEN_WORKER,empSt_15Q1_DEN_AVGSAL,empSt_15Q1_DEN_AVGPW,empSt_15Q1_DEN_TOPNAIC5,empSt_15Q1_DEN_TOPJOB,empSt_15Q1_DEN_TOPCITY,empSt_15Q1_DEN_PCTINSTATE,empSt_15Q1_WD_APP,empSt_15Q1_WD_WORKER,empSt_15Q1_WD_AVGSAL,empSt_15Q1_WD_AVGPW,empSt_15Q1_WD_TOPNAIC5,empSt_15Q1_WD_TOPJOB,empSt_15Q1_WD_TOPCITY,empSt_15Q1_WD_PCTINSTATE,empSt_15Q2_CERT_APP,empSt_15Q2_CERT_WORKER,empSt_15Q2_CERT_AVGSAL,empSt_15Q2_CERT_AVGPW,empSt_15Q2_CERT_TOPNAIC5,empSt_15Q2_CERT_TOPJOB,empSt_15Q2_CERT_TOPCITY,empSt_15Q2_CERT_PCTINSTATE,empSt_15Q2_CERTWD_APP,empSt_15Q2_CERTWD_WORKER,empSt_15Q2_CERTWD_AVGSAL,empSt_15Q2_CERTWD_AVGPW,empSt_15Q2_CERTWD_TOPNAIC5,empSt_15Q2_CERTWD_TOPJOB,empSt_15Q2_CERTWD_TOPCITY,empSt_15Q2_CERTWD_PCTINSTATE,empSt_15Q2_DEN_APP,empSt_15Q2_DEN_WORKER,empSt_15Q2_DEN_AVGSAL,empSt_15Q2_DEN_AVGPW,empSt_15Q2_DEN_TOPNAIC5,empSt_15Q2_DEN_TOPJOB,empSt_15Q2_DEN_TOPCITY,empSt_15Q2_DEN_PCTINSTATE,empSt_15Q2_WD_APP,empSt_15Q2_WD_WORKER,empSt_15Q2_WD_AVGSAL,empSt_15Q2_WD_AVGPW,empSt_15Q2_WD_TOPNAIC5,empSt_15Q2_WD_TOPJOB,empSt_15Q2_WD_TOPCITY,empSt_15Q2_WD_PCTINSTATE,empSt_15Q3_CERT_APP,empSt_15Q3_CERT_WORKER,empSt_15Q3_CERT_AVGSAL,empSt_15Q3_CERT_AVGPW,empSt_15Q3_CERT_TOPNAIC5,empSt_15Q3_CERT_TOPJOB,empSt_15Q3_CERT_TOPCITY,empSt_15Q3_CERT_PCTINSTATE,empSt_15Q3_CERTWD_APP,empSt_15Q3_CERTWD_WORKER,empSt_15Q3_CERTWD_AVGSAL,empSt_15Q3_CERTWD_AVGPW,empSt_15Q3_CERTWD_TOPNAIC5,empSt_15Q3_CERTWD_TOPJOB,empSt_15Q3_CERTWD_TOPCITY,empSt_15Q3_CERTWD_PCTINSTATE,empSt_15Q3_DEN_APP,empSt_15Q3_DEN_WORKER,empSt_15Q3_DEN_AVGSAL,empSt_15Q3_DEN_AVGPW,empSt_15Q3_DEN_TOPNAIC5,empSt_15Q3_DEN_TOPJOB,empSt_15Q3_DEN_TOPCITY,empSt_15Q3_DEN_PCTINSTATE,empSt_15Q3_WD_APP,empSt_15Q3_WD_WORKER,empSt_15Q3_WD_AVGSAL,empSt_15Q3_WD_AVGPW,empSt_15Q3_WD_TOPNAIC5,empSt_15Q3_WD_TOPJOB,empSt_15Q3_WD_TOPCITY,empSt_15Q3_WD_PCTINSTATE,empSt_15Q4_CERT_APP,empSt_15Q4_CERT_WORKER,empSt_15Q4_CERT_AVGSAL,empSt_15Q4_CERT_AVGPW,empSt_15Q4_CERT_TOPNAIC5,empSt_15Q4_CERT_TOPJOB,empSt_15Q4_CERT_TOPCITY,empSt_15Q4_CERT_PCTINSTATE,empSt_15Q4_CERTWD_APP,empSt_15Q4_CERTWD_WORKER,empSt_15Q4_CERTWD_AVGSAL,empSt_15Q4_CERTWD_AVGPW,empSt_15Q4_CERTWD_TOPNAIC5,empSt_15Q4_CERTWD_TOPJOB,empSt_15Q4_CERTWD_TOPCITY,empSt_15Q4_CERTWD_PCTINSTATE,empSt_15Q4_DEN_APP,empSt_15Q4_DEN_WORKER,empSt_15Q4_DEN_AVGSAL,empSt_15Q4_DEN_AVGPW,empSt_15Q4_DEN_TOPNAIC5,empSt_15Q4_DEN_TOPJOB,empSt_15Q4_DEN_TOPCITY,empSt_15Q4_DEN_PCTINSTATE,empSt_15Q4_WD_APP,empSt_15Q4_WD_WORKER,empSt_15Q4_WD_AVGSAL,empSt_15Q4_WD_AVGPW,empSt_15Q4_WD_TOPNAIC5,empSt_15Q4_WD_TOPJOB,empSt_15Q4_WD_TOPCITY,empSt_15Q4_WD_PCTINSTATE,empSt_16Q1_CERT_APP,empSt_16Q1_CERT_WORKER,empSt_16Q1_CERT_AVGSAL,empSt_16Q1_CERT_AVGPW,empSt_16Q1_CERT_TOPNAIC5,empSt_16Q1_CERT_TOPJOB,empSt_16Q1_CERT_TOPCITY,empSt_16Q1_CERT_PCTINSTATE,empSt_16Q1_CERTWD_APP,empSt_16Q1_CERTWD_WORKER,empSt_16Q1_CERTWD_AVGSAL,empSt_16Q1_CERTWD_AVGPW,empSt_16Q1_CERTWD_TOPNAIC5,empSt_16Q1_CERTWD_TOPJOB,empSt_16Q1_CERTWD_TOPCITY,empSt_16Q1_CERTWD_PCTINSTATE,empSt_16Q1_DEN_APP,empSt_16Q1_DEN_WORKER,empSt_16Q1_DEN_AVGSAL,empSt_16Q1_DEN_AVGPW,empSt_16Q1_DEN_TOPNAIC5,empSt_16Q1_DEN_TOPJOB,empSt_16Q1_DEN_TOPCITY,empSt_16Q1_DEN_PCTINSTATE,empSt_16Q1_WD_APP,empSt_16Q1_WD_WORKER,empSt_16Q1_WD_AVGSAL,empSt_16Q1_WD_AVGPW,empSt_16Q1_WD_TOPNAIC5,empSt_16Q1_WD_TOPJOB,empSt_16Q1_WD_TOPCITY,empSt_16Q1_WD_PCTINSTATE,empSt_16Q2_CERT_APP,empSt_16Q2_CERT_WORKER,empSt_16Q2_CERT_AVGSAL,empSt_16Q2_CERT_AVGPW,empSt_16Q2_CERT_TOPNAIC5,empSt_16Q2_CERT_TOPJOB,empSt_16Q2_CERT_TOPCITY,empSt_16Q2_CERT_PCTINSTATE,empSt_16Q2_CERTWD_APP,empSt_16Q2_CERTWD_WORKER,empSt_16Q2_CERTWD_AVGSAL,empSt_16Q2_CERTWD_AV

In [58]:
byState_worksite = pd.merge(statesj, workSt_final, how = 'left', on = 'STATE')
byState_worksite = byState_employer.to_crs({'init': 'epsg:4326'})
byState_worksite.head()

,STATE,NAME,geometry,empSt_15Q1_CERT_APP,empSt_15Q1_CERT_WORKER,empSt_15Q1_CERT_AVGSAL,empSt_15Q1_CERT_AVGPW,empSt_15Q1_CERT_TOPNAIC5,empSt_15Q1_CERT_TOPJOB,empSt_15Q1_CERT_TOPCITY,empSt_15Q1_CERT_PCTINSTATE,empSt_15Q1_CERTWD_APP,empSt_15Q1_CERTWD_WORKER,empSt_15Q1_CERTWD_AVGSAL,empSt_15Q1_CERTWD_AVGPW,empSt_15Q1_CERTWD_TOPNAIC5,empSt_15Q1_CERTWD_TOPJOB,empSt_15Q1_CERTWD_TOPCITY,empSt_15Q1_CERTWD_PCTINSTATE,empSt_15Q1_DEN_APP,empSt_15Q1_DEN_WORKER,empSt_15Q1_DEN_AVGSAL,empSt_15Q1_DEN_AVGPW,empSt_15Q1_DEN_TOPNAIC5,empSt_15Q1_DEN_TOPJOB,empSt_15Q1_DEN_TOPCITY,empSt_15Q1_DEN_PCTINSTATE,empSt_15Q1_WD_APP,empSt_15Q1_WD_WORKER,empSt_15Q1_WD_AVGSAL,empSt_15Q1_WD_AVGPW,empSt_15Q1_WD_TOPNAIC5,empSt_15Q1_WD_TOPJOB,empSt_15Q1_WD_TOPCITY,empSt_15Q1_WD_PCTINSTATE,empSt_15Q2_CERT_APP,empSt_15Q2_CERT_WORKER,empSt_15Q2_CERT_AVGSAL,empSt_15Q2_CERT_AVGPW,empSt_15Q2_CERT_TOPNAIC5,empSt_15Q2_CERT_TOPJOB,empSt_15Q2_CERT_TOPCITY,empSt_15Q2_CERT_PCTINSTATE,empSt_15Q2_CERTWD_APP,empSt_15Q2_CERTWD_WORKER,empSt_15Q2_CERTWD_AVGSAL,empSt_15Q2_CERTWD_AVGPW,empSt_15Q2_CERTWD_TOPNAIC5,empSt_15Q2_CERTWD_TOPJOB,empSt_15Q2_CERTWD_TOPCITY,empSt_15Q2_CERTWD_PCTINSTATE,empSt_15Q2_DEN_APP,empSt_15Q2_DEN_WORKER,empSt_15Q2_DEN_AVGSAL,empSt_15Q2_DEN_AVGPW,empSt_15Q2_DEN_TOPNAIC5,empSt_15Q2_DEN_TOPJOB,empSt_15Q2_DEN_TOPCITY,empSt_15Q2_DEN_PCTINSTATE,empSt_15Q2_WD_APP,empSt_15Q2_WD_WORKER,empSt_15Q2_WD_AVGSAL,empSt_15Q2_WD_AVGPW,empSt_15Q2_WD_TOPNAIC5,empSt_15Q2_WD_TOPJOB,empSt_15Q2_WD_TOPCITY,empSt_15Q2_WD_PCTINSTATE,empSt_15Q3_CERT_APP,empSt_15Q3_CERT_WORKER,empSt_15Q3_CERT_AVGSAL,empSt_15Q3_CERT_AVGPW,empSt_15Q3_CERT_TOPNAIC5,empSt_15Q3_CERT_TOPJOB,empSt_15Q3_CERT_TOPCITY,empSt_15Q3_CERT_PCTINSTATE,empSt_15Q3_CERTWD_APP,empSt_15Q3_CERTWD_WORKER,empSt_15Q3_CERTWD_AVGSAL,empSt_15Q3_CERTWD_AVGPW,empSt_15Q3_CERTWD_TOPNAIC5,empSt_15Q3_CERTWD_TOPJOB,empSt_15Q3_CERTWD_TOPCITY,empSt_15Q3_CERTWD_PCTINSTATE,empSt_15Q3_DEN_APP,empSt_15Q3_DEN_WORKER,empSt_15Q3_DEN_AVGSAL,empSt_15Q3_DEN_AVGPW,empSt_15Q3_DEN_TOPNAIC5,empSt_15Q3_DEN_TOPJOB,empSt_15Q3_DEN_TOPCITY,empSt_15Q3_DEN_PCTINSTATE,empSt_15Q3_WD_APP,empSt_15Q3_WD_WORKER,empSt_15Q3_WD_AVGSAL,empSt_15Q3_WD_AVGPW,empSt_15Q3_WD_TOPNAIC5,empSt_15Q3_WD_TOPJOB,empSt_15Q3_WD_TOPCITY,empSt_15Q3_WD_PCTINSTATE,empSt_15Q4_CERT_APP,empSt_15Q4_CERT_WORKER,empSt_15Q4_CERT_AVGSAL,empSt_15Q4_CERT_AVGPW,empSt_15Q4_CERT_TOPNAIC5,empSt_15Q4_CERT_TOPJOB,empSt_15Q4_CERT_TOPCITY,empSt_15Q4_CERT_PCTINSTATE,empSt_15Q4_CERTWD_APP,empSt_15Q4_CERTWD_WORKER,empSt_15Q4_CERTWD_AVGSAL,empSt_15Q4_CERTWD_AVGPW,empSt_15Q4_CERTWD_TOPNAIC5,empSt_15Q4_CERTWD_TOPJOB,empSt_15Q4_CERTWD_TOPCITY,empSt_15Q4_CERTWD_PCTINSTATE,empSt_15Q4_DEN_APP,empSt_15Q4_DEN_WORKER,empSt_15Q4_DEN_AVGSAL,empSt_15Q4_DEN_AVGPW,empSt_15Q4_DEN_TOPNAIC5,empSt_15Q4_DEN_TOPJOB,empSt_15Q4_DEN_TOPCITY,empSt_15Q4_DEN_PCTINSTATE,empSt_15Q4_WD_APP,empSt_15Q4_WD_WORKER,empSt_15Q4_WD_AVGSAL,empSt_15Q4_WD_AVGPW,empSt_15Q4_WD_TOPNAIC5,empSt_15Q4_WD_TOPJOB,empSt_15Q4_WD_TOPCITY,empSt_15Q4_WD_PCTINSTATE,empSt_16Q1_CERT_APP,empSt_16Q1_CERT_WORKER,empSt_16Q1_CERT_AVGSAL,empSt_16Q1_CERT_AVGPW,empSt_16Q1_CERT_TOPNAIC5,empSt_16Q1_CERT_TOPJOB,empSt_16Q1_CERT_TOPCITY,empSt_16Q1_CERT_PCTINSTATE,empSt_16Q1_CERTWD_APP,empSt_16Q1_CERTWD_WORKER,empSt_16Q1_CERTWD_AVGSAL,empSt_16Q1_CERTWD_AVGPW,empSt_16Q1_CERTWD_TOPNAIC5,empSt_16Q1_CERTWD_TOPJOB,empSt_16Q1_CERTWD_TOPCITY,empSt_16Q1_CERTWD_PCTINSTATE,empSt_16Q1_DEN_APP,empSt_16Q1_DEN_WORKER,empSt_16Q1_DEN_AVGSAL,empSt_16Q1_DEN_AVGPW,empSt_16Q1_DEN_TOPNAIC5,empSt_16Q1_DEN_TOPJOB,empSt_16Q1_DEN_TOPCITY,empSt_16Q1_DEN_PCTINSTATE,empSt_16Q1_WD_APP,empSt_16Q1_WD_WORKER,empSt_16Q1_WD_AVGSAL,empSt_16Q1_WD_AVGPW,empSt_16Q1_WD_TOPNAIC5,empSt_16Q1_WD_TOPJOB,empSt_16Q1_WD_TOPCITY,empSt_16Q1_WD_PCTINSTATE,empSt_16Q2_CERT_APP,empSt_16Q2_CERT_WORKER,empSt_16Q2_CERT_AVGSAL,empSt_16Q2_CERT_AVGPW,empSt_16Q2_CERT_TOPNAIC5,empSt_16Q2_CERT_TOPJOB,empSt_16Q2_CERT_TOPCITY,empSt_16Q2_CERT_PCTINSTATE,empSt_16Q2_CERTWD_APP,empSt_16Q2_CERTWD_WORKER,empSt_16Q2_CERTWD_AVGSAL,empSt_16Q2_CERTWD_AV

In [59]:
countiesj = counties[['GEOID', 'NAME', 'geometry']].copy()
countiesj.columns = ['CNTY', 'NAME', 'geometry']
countiesj.head()

,CNTY,NAME,geometry
0,04015,Mohave,POLYGON ((-12774536.96142742 4287452.834847284...
1,22105,Tangipahoa,POLYGON ((-10081891.35830733 3588125.959690343...
2,16063,Lincoln,POLYGON ((-12756613.07625632 5312952.585752421...
3,27119,Polk,POLYGON ((-10814318.05930117 6103575.795496427...
4,38017,Cass,POLYGON ((-10876585.95231028 5949946.354262205...


In [60]:
byCounty_employer = pd.merge(countiesj, emp_final, how = 'left', on = 'CNTY')
byCounty_employer = byCounty_employer.to_crs({'init': 'epsg:4326'})
byCounty_employer.head()

,CNTY,NAME,geometry,emp_15Q1_CERT_APP,emp_15Q1_CERT_WORKER,emp_15Q1_CERT_AVGSAL,emp_15Q1_CERT_AVGPW,emp_15Q1_CERT_TOPNAIC5,emp_15Q1_CERT_TOPJOB,emp_15Q1_CERT_TOPCITY,emp_15Q1_CERT_PCTINSTATE,emp_15Q1_CERTWD_APP,emp_15Q1_CERTWD_WORKER,emp_15Q1_CERTWD_AVGSAL,emp_15Q1_CERTWD_AVGPW,emp_15Q1_CERTWD_TOPNAIC5,emp_15Q1_CERTWD_TOPJOB,emp_15Q1_CERTWD_TOPCITY,emp_15Q1_CERTWD_PCTINSTATE,emp_15Q1_DEN_APP,emp_15Q1_DEN_WORKER,emp_15Q1_DEN_AVGSAL,emp_15Q1_DEN_AVGPW,emp_15Q1_DEN_TOPNAIC5,emp_15Q1_DEN_TOPJOB,emp_15Q1_DEN_TOPCITY,emp_15Q1_DEN_PCTINSTATE,emp_15Q1_WD_APP,emp_15Q1_WD_WORKER,emp_15Q1_WD_AVGSAL,emp_15Q1_WD_AVGPW,emp_15Q1_WD_TOPNAIC5,emp_15Q1_WD_TOPJOB,emp_15Q1_WD_TOPCITY,emp_15Q1_WD_PCTINSTATE,emp_15Q2_CERT_APP,emp_15Q2_CERT_WORKER,emp_15Q2_CERT_AVGSAL,emp_15Q2_CERT_AVGPW,emp_15Q2_CERT_TOPNAIC5,emp_15Q2_CERT_TOPJOB,emp_15Q2_CERT_TOPCITY,emp_15Q2_CERT_PCTINSTATE,emp_15Q2_CERTWD_APP,emp_15Q2_CERTWD_WORKER,emp_15Q2_CERTWD_AVGSAL,emp_15Q2_CERTWD_AVGPW,emp_15Q2_CERTWD_TOPNAIC5,emp_15Q2_CERTWD_TOPJOB,emp_15Q2_CERTWD_TOPCITY,emp_15Q2_CERTWD_PCTINSTATE,emp_15Q2_DEN_APP,emp_15Q2_DEN_WORKER,emp_15Q2_DEN_AVGSAL,emp_15Q2_DEN_AVGPW,emp_15Q2_DEN_TOPNAIC5,emp_15Q2_DEN_TOPJOB,emp_15Q2_DEN_TOPCITY,emp_15Q2_DEN_PCTINSTATE,emp_15Q2_WD_APP,emp_15Q2_WD_WORKER,emp_15Q2_WD_AVGSAL,emp_15Q2_WD_AVGPW,emp_15Q2_WD_TOPNAIC5,emp_15Q2_WD_TOPJOB,emp_15Q2_WD_TOPCITY,emp_15Q2_WD_PCTINSTATE,emp_15Q3_CERT_APP,emp_15Q3_CERT_WORKER,emp_15Q3_CERT_AVGSAL,emp_15Q3_CERT_AVGPW,emp_15Q3_CERT_TOPNAIC5,emp_15Q3_CERT_TOPJOB,emp_15Q3_CERT_TOPCITY,emp_15Q3_CERT_PCTINSTATE,emp_15Q3_CERTWD_APP,emp_15Q3_CERTWD_WORKER,emp_15Q3_CERTWD_AVGSAL,emp_15Q3_CERTWD_AVGPW,emp_15Q3_CERTWD_TOPNAIC5,emp_15Q3_CERTWD_TOPJOB,emp_15Q3_CERTWD_TOPCITY,emp_15Q3_CERTWD_PCTINSTATE,emp_15Q3_DEN_APP,emp_15Q3_DEN_WORKER,emp_15Q3_DEN_AVGSAL,emp_15Q3_DEN_AVGPW,emp_15Q3_DEN_TOPNAIC5,emp_15Q3_DEN_TOPJOB,emp_15Q3_DEN_TOPCITY,emp_15Q3_DEN_PCTINSTATE,emp_15Q3_WD_APP,emp_15Q3_WD_WORKER,emp_15Q3_WD_AVGSAL,emp_15Q3_WD_AVGPW,emp_15Q3_WD_TOPNAIC5,emp_15Q3_WD_TOPJOB,emp_15Q3_WD_TOPCITY,emp_15Q3_WD_PCTINSTATE,emp_15Q4_CERT_APP,emp_15Q4_CERT_WORKER,emp_15Q4_CERT_AVGSAL,emp_15Q4_CERT_AVGPW,emp_15Q4_CERT_TOPNAIC5,emp_15Q4_CERT_TOPJOB,emp_15Q4_CERT_TOPCITY,emp_15Q4_CERT_PCTINSTATE,emp_15Q4_CERTWD_APP,emp_15Q4_CERTWD_WORKER,emp_15Q4_CERTWD_AVGSAL,emp_15Q4_CERTWD_AVGPW,emp_15Q4_CERTWD_TOPNAIC5,emp_15Q4_CERTWD_TOPJOB,emp_15Q4_CERTWD_TOPCITY,emp_15Q4_CERTWD_PCTINSTATE,emp_15Q4_DEN_APP,emp_15Q4_DEN_WORKER,emp_15Q4_DEN_AVGSAL,emp_15Q4_DEN_AVGPW,emp_15Q4_DEN_TOPNAIC5,emp_15Q4_DEN_TOPJOB,emp_15Q4_DEN_TOPCITY,emp_15Q4_DEN_PCTINSTATE,emp_15Q4_WD_APP,emp_15Q4_WD_WORKER,emp_15Q4_WD_AVGSAL,emp_15Q4_WD_AVGPW,emp_15Q4_WD_TOPNAIC5,emp_15Q4_WD_TOPJOB,emp_15Q4_WD_TOPCITY,emp_15Q4_WD_PCTINSTATE,emp_16Q1_CERT_APP,emp_16Q1_CERT_WORKER,emp_16Q1_CERT_AVGSAL,emp_16Q1_CERT_AVGPW,emp_16Q1_CERT_TOPNAIC5,emp_16Q1_CERT_TOPJOB,emp_16Q1_CERT_TOPCITY,emp_16Q1_CERT_PCTINSTATE,emp_16Q1_CERTWD_APP,emp_16Q1_CERTWD_WORKER,emp_16Q1_CERTWD_AVGSAL,emp_16Q1_CERTWD_AVGPW,emp_16Q1_CERTWD_TOPNAIC5,emp_16Q1_CERTWD_TOPJOB,emp_16Q1_CERTWD_TOPCITY,emp_16Q1_CERTWD_PCTINSTATE,emp_16Q1_DEN_APP,emp_16Q1_DEN_WORKER,emp_16Q1_DEN_AVGSAL,emp_16Q1_DEN_AVGPW,emp_16Q1_DEN_TOPNAIC5,emp_16Q1_DEN_TOPJOB,emp_16Q1_DEN_TOPCITY,emp_16Q1_DEN_PCTINSTATE,emp_16Q1_WD_APP,emp_16Q1_WD_WORKER,emp_16Q1_WD_AVGSAL,emp_16Q1_WD_AVGPW,emp_16Q1_WD_TOPNAIC5,emp_16Q1_WD_TOPJOB,emp_16Q1_WD_TOPCITY,emp_16Q1_WD_PCTINSTATE,emp_16Q2_CERT_APP,emp_16Q2_CERT_WORKER,emp_16Q2_CERT_AVGSAL,emp_16Q2_CERT_AVGPW,emp_16Q2_CERT_TOPNAIC5,emp_16Q2_CERT_TOPJOB,emp_16Q2_CERT_TOPCITY,emp_16Q2_CERT_PCTINSTATE,emp_16Q2_CERTWD_APP,emp_16Q2_CERTWD_WORKER,emp_16Q2_CERTWD_AVGSAL,emp_16Q2_CERTWD_AVGPW,emp_16Q2_CERTWD_TOPNAIC5,emp_16Q2_CERTWD_TOPJOB,emp_16Q2_CERTWD_TOPCITY,emp_16Q2_CERTWD_PCTINSTATE,emp_16Q2_DEN_APP,emp_16Q2_DEN_WORKER,emp_16Q2_DEN_AVGSAL,emp_16Q2_DEN_AVGPW,emp_16Q2_DEN_TOPNAIC5,emp_16Q2_DEN_TOPJOB,emp_16Q2_DEN_TOPCITY,emp_16Q2_DEN_PCTINSTATE,emp_16Q2_WD_APP,emp_16Q2_WD_WORKER,emp_16Q2_WD_AVGSAL,emp_16Q2_WD_AVGPW,emp_16Q

In [61]:
byCounty_worksite = pd.merge(countiesj, work_final, how = 'left', on = 'CNTY')
byCounty_worksite = byCounty_worksite.to_crs({'init': 'epsg:4326'})
byCounty_worksite.head()

,CNTY,NAME,geometry,work_15Q1_CERT_APP,work_15Q1_CERT_WORKER,work_15Q1_CERT_AVGSAL,work_15Q1_CERT_AVGPW,work_15Q1_CERT_TOPNAIC5,work_15Q1_CERT_TOPJOB,work_15Q1_CERT_TOPCITY,work_15Q1_CERT_PCTINSTATE,work_15Q1_CERTWD_APP,work_15Q1_CERTWD_WORKER,work_15Q1_CERTWD_AVGSAL,work_15Q1_CERTWD_AVGPW,work_15Q1_CERTWD_TOPNAIC5,work_15Q1_CERTWD_TOPJOB,work_15Q1_CERTWD_TOPCITY,work_15Q1_CERTWD_PCTINSTATE,work_15Q1_DEN_APP,work_15Q1_DEN_WORKER,work_15Q1_DEN_AVGSAL,work_15Q1_DEN_AVGPW,work_15Q1_DEN_TOPNAIC5,work_15Q1_DEN_TOPJOB,work_15Q1_DEN_TOPCITY,work_15Q1_DEN_PCTINSTATE,work_15Q1_WD_APP,work_15Q1_WD_WORKER,work_15Q1_WD_AVGSAL,work_15Q1_WD_AVGPW,work_15Q1_WD_TOPNAIC5,work_15Q1_WD_TOPJOB,work_15Q1_WD_TOPCITY,work_15Q1_WD_PCTINSTATE,work_15Q2_CERT_APP,work_15Q2_CERT_WORKER,work_15Q2_CERT_AVGSAL,work_15Q2_CERT_AVGPW,work_15Q2_CERT_TOPNAIC5,work_15Q2_CERT_TOPJOB,work_15Q2_CERT_TOPCITY,work_15Q2_CERT_PCTINSTATE,work_15Q2_CERTWD_APP,work_15Q2_CERTWD_WORKER,work_15Q2_CERTWD_AVGSAL,work_15Q2_CERTWD_AVGPW,work_15Q2_CERTWD_TOPNAIC5,work_15Q2_CERTWD_TOPJOB,work_15Q2_CERTWD_TOPCITY,work_15Q2_CERTWD_PCTINSTATE,work_15Q2_DEN_APP,work_15Q2_DEN_WORKER,work_15Q2_DEN_AVGSAL,work_15Q2_DEN_AVGPW,work_15Q2_DEN_TOPNAIC5,work_15Q2_DEN_TOPJOB,work_15Q2_DEN_TOPCITY,work_15Q2_DEN_PCTINSTATE,work_15Q2_WD_APP,work_15Q2_WD_WORKER,work_15Q2_WD_AVGSAL,work_15Q2_WD_AVGPW,work_15Q2_WD_TOPNAIC5,work_15Q2_WD_TOPJOB,work_15Q2_WD_TOPCITY,work_15Q2_WD_PCTINSTATE,work_15Q3_CERT_APP,work_15Q3_CERT_WORKER,work_15Q3_CERT_AVGSAL,work_15Q3_CERT_AVGPW,work_15Q3_CERT_TOPNAIC5,work_15Q3_CERT_TOPJOB,work_15Q3_CERT_TOPCITY,work_15Q3_CERT_PCTINSTATE,work_15Q3_CERTWD_APP,work_15Q3_CERTWD_WORKER,work_15Q3_CERTWD_AVGSAL,work_15Q3_CERTWD_AVGPW,work_15Q3_CERTWD_TOPNAIC5,work_15Q3_CERTWD_TOPJOB,work_15Q3_CERTWD_TOPCITY,work_15Q3_CERTWD_PCTINSTATE,work_15Q3_DEN_APP,work_15Q3_DEN_WORKER,work_15Q3_DEN_AVGSAL,work_15Q3_DEN_AVGPW,work_15Q3_DEN_TOPNAIC5,work_15Q3_DEN_TOPJOB,work_15Q3_DEN_TOPCITY,work_15Q3_DEN_PCTINSTATE,work_15Q3_WD_APP,work_15Q3_WD_WORKER,work_15Q3_WD_AVGSAL,work_15Q3_WD_AVGPW,work_15Q3_WD_TOPNAIC5,work_15Q3_WD_TOPJOB,work_15Q3_WD_TOPCITY,work_15Q3_WD_PCTINSTATE,work_15Q4_CERT_APP,work_15Q4_CERT_WORKER,work_15Q4_CERT_AVGSAL,work_15Q4_CERT_AVGPW,work_15Q4_CERT_TOPNAIC5,work_15Q4_CERT_TOPJOB,work_15Q4_CERT_TOPCITY,work_15Q4_CERT_PCTINSTATE,work_15Q4_CERTWD_APP,work_15Q4_CERTWD_WORKER,work_15Q4_CERTWD_AVGSAL,work_15Q4_CERTWD_AVGPW,work_15Q4_CERTWD_TOPNAIC5,work_15Q4_CERTWD_TOPJOB,work_15Q4_CERTWD_TOPCITY,work_15Q4_CERTWD_PCTINSTATE,work_15Q4_DEN_APP,work_15Q4_DEN_WORKER,work_15Q4_DEN_AVGSAL,work_15Q4_DEN_AVGPW,work_15Q4_DEN_TOPNAIC5,work_15Q4_DEN_TOPJOB,work_15Q4_DEN_TOPCITY,work_15Q4_DEN_PCTINSTATE,work_15Q4_WD_APP,work_15Q4_WD_WORKER,work_15Q4_WD_AVGSAL,work_15Q4_WD_AVGPW,work_15Q4_WD_TOPNAIC5,work_15Q4_WD_TOPJOB,work_15Q4_WD_TOPCITY,work_15Q4_WD_PCTINSTATE,work_16Q1_CERT_APP,work_16Q1_CERT_WORKER,work_16Q1_CERT_AVGSAL,work_16Q1_CERT_AVGPW,work_16Q1_CERT_TOPNAIC5,work_16Q1_CERT_TOPJOB,work_16Q1_CERT_TOPCITY,work_16Q1_CERT_PCTINSTATE,work_16Q1_CERTWD_APP,work_16Q1_CERTWD_WORKER,work_16Q1_CERTWD_AVGSAL,work_16Q1_CERTWD_AVGPW,work_16Q1_CERTWD_TOPNAIC5,work_16Q1_CERTWD_TOPJOB,work_16Q1_CERTWD_TOPCITY,work_16Q1_CERTWD_PCTINSTATE,work_16Q1_DEN_APP,work_16Q1_DEN_WORKER,work_16Q1_DEN_AVGSAL,work_16Q1_DEN_AVGPW,work_16Q1_DEN_TOPNAIC5,work_16Q1_DEN_TOPJOB,work_16Q1_DEN_TOPCITY,work_16Q1_DEN_PCTINSTATE,work_16Q1_WD_APP,work_16Q1_WD_WORKER,work_16Q1_WD_AVGSAL,work_16Q1_WD_AVGPW,work_16Q1_WD_TOPNAIC5,work_16Q1_WD_TOPJOB,work_16Q1_WD_TOPCITY,work_16Q1_WD_PCTINSTATE,work_16Q2_CERT_APP,work_16Q2_CERT_WORKER,work_16Q2_CERT_AVGSAL,work_16Q2_CERT_AVGPW,work_16Q2_CERT_TOPNAIC5,work_16Q2_CERT_TOPJOB,work_16Q2_CERT_TOPCITY,work_16Q2_CERT_PCTINSTATE,work_16Q2_CERTWD_APP,work_16Q2_CERTWD_WORKER,work_16Q2_CERTWD_AVGSAL,work_16Q2_CERTWD_AVGPW,work_16Q2_CERTWD_TOPNAIC5,work_16Q2_CERTWD_TOPJOB,work_16Q2_CERTWD_TOPCITY,work_16Q2_CERTWD_PCTINSTATE,work_16Q2_DEN_APP,work_16Q2_DEN_WORKER,work_16Q2_DEN_AVGSAL,work_1

In [880]:
byState_worksite.to_file("../newDat/byState_worksite.geojson", driver = 'GeoJSON')

In [890]:
byCounty_worksite.to_file("../newDat/byCounty_worksite.geojson", driver = 'GeoJSON')

In [888]:
byCounty_employer.to_file("../newDat/byCounty_employer.geojson", driver = 'GeoJSON')

In [877]:
#byState_employer.to_file("../newDat/byState_employer.geojson", driver = 'GeoJSON')
byState_employer.to_file("../newDat/finalmash/byState_employer.csv", driver = 'GeoJSON')

## REMEMBER before exporting: 
* change float to int by pd.set_option('precision', 0)
* change nan to NaN for geojson

In [868]:
import fiona; fiona.supported_drivers

{'ARCGEN': 'r',
 'AeronavFAA': 'r',
 'BNA': 'raw',
 'DGN': 'raw',
 'DXF': 'raw',
 'ESRI Shapefile': 'raw',
 'GPKG': 'rw',
 'GPSTrackMaker': 'raw',
 'GPX': 'raw',
 'GeoJSON': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'OpenFileGDB': 'r',
 'PCIDSK': 'r',
 'SEGY': 'r',
 'SUA': 'r'}

In [ ]:
gdf.to_file("test.geojson", driver='GeoJSON')

In [891]:
highlightcities = pd.read_csv("../newDat/miscignore/fillCityList_ffinal.csv")

In [908]:
highlightcities.tail(20)

,MATCHKEY,CITYLABEL,FREQ,LAT,LNG,TYPE,STATEEMP,STATEWORK,NATEMP,NATWORK
513,IRVING-TX,"IRVING, TX",90,32.830,-96.944,"National Top Employer, National Top Employer, ...",True,False,True,True
514,TROY-MI,"TROY, MI",91,42.606,-83.150,"National Top Employer, National Top Employer, ...",True,False,True,True
515,BOISE-ID,"BOISE, ID",92,43.617,-116.201,"State Top Employer, State Top Employer, State ...",True,False,False,False
516,OMAHA-NE,"OMAHA, NE",95,41.259,-95.938,"State Top Employer, State Top Employer, State ...",True,False,False,False
517,DURHAM-NC,"DURHAM, NC",95,35.997,-78.902,"National Top Employer, National Top Employer, ...",True,False,True,False
518,LOUISVILLE-KY,"LOUISVILLE, KY",98,38.254,-85.759,"National Top Employer, National Top Employer, ...",True,False,True,False
519,LAS VEGAS-NV,"LAS VEGAS, NV",101,36.166,-115.149,"State Top Employer, State Top Employer, State ...",True,False,False,False
520,HONOLULU-HI,"HONOLULU, HI",102,21.325,-157.847,"National Top Worksite, State Top Employer, Sta...",True,False,False,True
521,MOUNTAIN VIEW-CA,"MOUNTAIN VIEW, CA",105,45.507,-73.605,"National Top Employer, National Top Employer, ...",True,False,True,True
522,PHOENIX-AZ,"PHOENIX, AZ",107,33.449,-112.077,"National Top Worksite, National Top Worksite, ...",True,False,False,True


In [900]:
highlightcities['top_state_emp'] = highlightcities['type'].str.contains("State Top Employer")
highlightcities['top_state_work'] = highlightcities['type'].str.contains("State Top Worksite")
highlightcities['top_nat_emp'] = highlightcities['type'].str.contains("National Top Employer")
highlightcities['top_nat_work'] = highlightcities['type'].str.contains("National Top Worksite")

In [906]:
highlightcities.columns = ['drop', 'MATCHKEY', 'CITYLABEL', 'FREQ', 'LAT', 'LNG', 'TYPE', 'STATEEMP', 'STATEWORK', 'NATEMP', 'NATWORK']
highlightcities.drop(columns = ['drop'], inplace = True)

In [907]:
highlightcities.head()

,MATCHKEY,CITYLABEL,FREQ,LAT,LNG,TYPE,STATEEMP,STATEWORK,NATEMP,NATWORK
0,SKOWHEGAN-ME,"SKOWHEGAN, ME",1,44.765,-69.719,State Top Employer,True,False,False,False
1,ELDORA-IA,"ELDORA, IA",1,42.361,-93.100,State Top Employer,True,False,False,False
2,ELIZABETHTOWN-KY,"ELIZABETHTOWN, KY",1,37.694,-85.859,State Top Employer,True,False,False,False
3,ELLINGTON-CT,"ELLINGTON, CT",1,41.904,-72.470,State Top Employer,True,False,False,False
4,NORTH MIAMI BEACH-FL,"NORTH MIAMI BEACH, FL",1,25.933,-80.163,National Top Worksite,False,False,False,True


In [909]:
highlightcities.to_csv("../newDat/highlightCities.csv", index = False)